# Transforming the Landing Layer to the Raw Layer

In [1]:
from pyspark.sql import SparkSession, functions as F

# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("ADS Project 1")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .getOrCreate()
)

your 131072x1 screen size is bogus. expect trouble
24/08/31 05:05:28 WARN Utils: Your hostname, LAPTOP-1A92TAQQ resolves to a loopback address: 127.0.1.1; using 172.18.166.45 instead (on interface eth0)
24/08/31 05:05:28 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/31 05:05:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/08/31 05:05:30 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/08/31 05:05:30 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


### Retrieving 2023 TLC Trip Records for Yellow Taxis

In [2]:
# from urllib.request import urlretrieve
import os

output_relative_dir = 'Landing/'

# check if it exists as it makedir will raise an error if it does exist
if not os.path.exists(output_relative_dir):
    os.makedirs(output_relative_dir)
    
# now, for each type of data set we will need, we will create the paths
if not os.path.exists(output_relative_dir + 'tlc_yellow_data'):
    os.makedirs(output_relative_dir + 'tlc_yellow_data')


In [3]:
from urllib.request import urlretrieve

# retrieve yellow and green taxi data for the year 2023
YEAR = '2023'
MONTHS = range(1, 13)

URL_TEMPLATE = "https://d37ci6vzurychx.cloudfront.net/trip-data/"#year-month.parquet

# data output directory is `data/tlc_data/`

for month in MONTHS:
    # 0-fill i.e 1 -> 01, 2 -> 02, etc
    month = str(month).zfill(2) 
    print(f"Begin month {month}")
    
    # generate url
    url = f'{URL_TEMPLATE}yellow_tripdata_{YEAR}-{month}.parquet'
    # generate output location and filename
    output_dir = f"{output_relative_dir}tlc_yellow_data/{YEAR}-{month}.parquet"
    # download
    urlretrieve(url, output_dir) 
    
    print(f"Completed month {month}")

Begin month 01
Completed month 01
Begin month 02
Completed month 02
Begin month 03
Completed month 03
Begin month 04
Completed month 04
Begin month 05
Completed month 05
Begin month 06
Completed month 06
Begin month 07
Completed month 07
Begin month 08
Completed month 08
Begin month 09
Completed month 09
Begin month 10
Completed month 10
Begin month 11
Completed month 11
Begin month 12
Completed month 12


### Fixing the Parquet for January 2023 Yellow Taxi Data

In [4]:
sdf_feb = spark.read.parquet('Landing/tlc_yellow_data/2023-02.parquet')

consistent_col_casing = [F.col(col_name).alias(col_name.lower()) for col_name in sdf_feb.columns]
sdf_feb = sdf_feb.select(*consistent_col_casing)

# this will be used in the cell below when reading in
sdf_schema = sdf_feb.schema

In [5]:
sdf_malformed = spark.read.parquet('Landing/tlc_yellow_data/2023-01.parquet')

# select all columns from the existing malformed dataframe and cast it to the required schema
sdf_malformed = sdf_malformed \
    .select([F.col(c).cast(sdf_schema[i].dataType) for i, c in enumerate(sdf_malformed.columns)])

In [6]:
sdf_malformed \
    .coalesce(1) \
    .write \
    .mode('overwrite') \
    .parquet('Raw/tlc_yellow_data/2023-01')

### Reading in all Yellow Taxi Data into a Folder

In [7]:
for month in range(2, 13):
    input_path = f'Landing/tlc_yellow_data/2023-{str(month).zfill(2)}.parquet'
    output_path = f'Raw/tlc_yellow_data/2023-{str(month).zfill(2)}'
    
    # read and write in one single statement, not great but it is what it is
    # do not copy this, this is not best practice
    sdf = spark \
        .read \
        .schema(sdf_schema) \
        .parquet(input_path) \
        .coalesce(1) \
        .write \
        .mode('overwrite') \
        .parquet(output_path)

In [8]:
sdf_yellow = spark.read.schema(sdf_schema).parquet('Raw/tlc_yellow_data/*')
sdf_yellow.limit(5)

vendorid,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,ratecodeid,store_and_fwd_flag,pulocationid,dolocationid,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
1,2023-05-01 00:33:13,2023-05-01 00:53:01,0,7.8,1,N,138,43,1,33.8,7.75,0.5,8.6,0.0,1.0,51.65,0.0,1.75
1,2023-05-01 00:42:49,2023-05-01 01:11:18,2,8.1,1,N,138,262,1,35.9,10.25,0.5,9.5,0.0,1.0,57.15,2.5,1.75
1,2023-05-01 00:56:34,2023-05-01 01:13:39,2,9.1,1,N,138,141,1,35.2,10.25,0.5,10.7,6.55,1.0,64.2,2.5,1.75
2,2023-05-01 00:00:52,2023-05-01 00:20:12,1,8.21,1,N,138,140,1,33.1,6.0,0.5,2.24,0.0,1.0,47.09,2.5,1.75
1,2023-05-01 00:05:50,2023-05-01 00:19:41,0,7.9,1,N,138,263,1,31.0,10.25,0.5,9.85,6.55,1.0,59.15,2.5,1.75


### Retrieving 2024 TLC Trip Records for Yellow Taxis

In [9]:
# from urllib.request import urlretrieve
import os

output_relative_dir = '2024_Raw/'

# check if it exists as it makedir will raise an error if it does exist
if not os.path.exists(output_relative_dir):
    os.makedirs(output_relative_dir)
    
# now, for each type of data set we will need, we will create the paths
if not os.path.exists(output_relative_dir + 'tlc_yellow_data'):
    os.makedirs(output_relative_dir + 'tlc_yellow_data')

In [10]:
from urllib.request import urlretrieve

# retrieve yellow and green taxi data for the year 2023
YEAR = '2024'
MONTHS = range(1, 7)

URL_TEMPLATE = "https://d37ci6vzurychx.cloudfront.net/trip-data/"#year-month.parquet

# data output directory is `data/tlc_data/`

for month in MONTHS:
    # 0-fill i.e 1 -> 01, 2 -> 02, etc
    month = str(month).zfill(2) 
    print(f"Begin month {month}")
    
    # generate url
    url = f'{URL_TEMPLATE}yellow_tripdata_{YEAR}-{month}.parquet'
    # generate output location and filename
    output_dir = f"{output_relative_dir}tlc_yellow_data/{YEAR}-{month}.parquet"
    # download
    urlretrieve(url, output_dir) 
    
    print(f"Completed month {month}")

Begin month 01
Completed month 01
Begin month 02
Completed month 02
Begin month 03
Completed month 03
Begin month 04
Completed month 04
Begin month 05
Completed month 05
Begin month 06
Completed month 06
